In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import tqdm

from scipy import sparse
import torch

import schrodingerUtils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

In [ ]:
NUM_DIMENSIONS = 1
POINTS_PER_DIMENSION = 10000

DIMENSION_LIMIT = 1
discreteSpatialMeshgrid = schrodingerUtils.physics.createNDimensionalMeshGrid(NUM_DIMENSIONS, POINTS_PER_DIMENSION, [-DIMENSION_LIMIT,DIMENSION_LIMIT])

In [ ]:
def createGaussian(mu, sigma):
    return lambda x: np.exp(-(x-mu)**2 / (2*sigma**2))

g1 = createGaussian(0.0, 0.1)
g2 = createGaussian(0.5, 0.1)
def potential(x):
    return - 1 * (1*g1(x) + 0*g2(x))

V = potential(*discreteSpatialMeshgrid)

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ax.plot(*discreteSpatialMeshgrid, V)
# plt.savefig("images/twoNarrowAttractors.png")
plt.show()

# Solve Time Independent Schrodinger Equation

In [ ]:
T = schrodingerUtils.physics.getKineticSchrodingerTerm(NUM_DIMENSIONS, POINTS_PER_DIMENSION)
U = sparse.diags(V.reshape(POINTS_PER_DIMENSION**NUM_DIMENSIONS), (0))

H = T+U

timeIndependentSolver = schrodingerUtils.physics.TorchTimeIndependentSchrodingerSolver(H, NUM_DIMENSIONS, POINTS_PER_DIMENSION)

In [ ]:
numEigenstates = 50
timeIndependentSolver.solveEigenstates(numEigenstates=numEigenstates)

In [ ]:
axisDim = 3

# numRows = 4
# numCols = numEigenstates//numRows
# numCols = 4
# numRows = numEigenstates//numCols
numCols = 4
numRows = 4


fig, axes = plt.subplots(ncols=numCols, nrows=numRows, sharex=True, sharey=True, figsize=(numCols*axisDim,numRows*axisDim))

axes = np.ravel(axes)
for i in range(len(axes)):
    targetAxis = axes[i]
    if i > numEigenstates:
        targetAxis.axis("off")
        continue

    targetAxis.set_title(f"Eigenstate Index: {i}")
    # targetAxis.set_aspect("equal")
    eigenvalue, eigenvector = timeIndependentSolver.getEigenstate(i)
    targetAxis.set_xlim([-0.1,0.1])
    targetAxis.plot(*discreteSpatialMeshgrid, eigenvector**2)
plt.tight_layout()
plt.show()

# Solve Time Dependent Schrodinger Equation

In [ ]:
TEMPORAL_SPACING = 1e-7
solver = schrodingerUtils.physics.TimeDependentSchrodingerSolver(V, discreteSpatialMeshgrid, TEMPORAL_SPACING)
solver.getDelTimeOverDelXSquare()

In [ ]:
g = createGaussian(0.3, 0.1)
initialPsi = g(discreteSpatialMeshgrid[0])
initialPsi = solver.normalizePsi(initialPsi)
plt.plot(*discreteSpatialMeshgrid, initialPsi)
plt.show()

In [ ]:
numTimesteps = 200000
psi = solver.solve(initialPsi, numTimesteps)

In [ ]:
plt.plot(*discreteSpatialMeshgrid, np.absolute(psi[-1])**2)
plt.show()

In [ ]:
stepsPerFrame = 200
numFrames = int(len(psi) / stepsPerFrame)
display(f"Total GIF time: {(numFrames/30):0.0f}s")

In [ ]:
framesProgressBar=tqdm.tqdm(total=numFrames, position=0, leave=True)

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ax.set_xlim((np.min(discreteSpatialMeshgrid[0]), np.max(discreteSpatialMeshgrid[0])))
ax.set_ylim(-1, 1.5*np.max(np.absolute(psi)**2))

l, = ax.plot([], [])

def animate(i):
    targetFrame = stepsPerFrame*i
    ax.set_title(f"{targetFrame:03} / {len(psi)}")
    l.set_data(*discreteSpatialMeshgrid, np.absolute(psi[targetFrame])**2)
    framesProgressBar.update(1)


# plt.tight_layout()
ani = animation.FuncAnimation(fig, animate, frames=numFrames, interval=50)
writer = animation.PillowWriter(fps=30,)
ani.save("images/psi.gif", writer=writer)
framesProgressBar.close()